In [1]:
#The version should be a more cleaned up version and will make final finishing touches
#get the dependencies that will be needed
import pandas as pd
#Also, for this project i feel confident the code will not result in anything breaking so i will trun off the settingwithcopy warning
pd.set_option('mode.chained_assignment', None)

In [2]:
#Load the raw data into the dataframes
school_data_raw = pd.read_csv("Resources/schools_complete.csv")
student_data_raw = pd.read_csv("Resources/students_complete.csv")

In [3]:
#Merged the dataframes together
school_data_complete = pd.merge(student_data_raw, school_data_raw, how="left", on=["school_name", "school_name"])

In [4]:
#Taking a look at the data. 
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [5]:
#using .unique() will give us the number of highschools in our data frame since there should not be any duplicate schools
number_of_schools = len(school_data_complete['school_name'].unique())

#For students we cannot assume there will not be duplicate names, but luckily each row is a unique student
number_of_students = len(school_data_complete['student_name'])

#To find the sum of the budget, we can just take the total from the budget column in the school dataframe
sum_budget = school_data_raw['budget'].sum()

#Finding the average score is simple using the .mean() function on the respective columns
average_math_score = school_data_complete["math_score"].mean()
average_reading_score = school_data_complete["reading_score"].mean()
average_overall_score = (average_math_score+average_reading_score)/2

In [6]:
#Now we need to find the percentage of passing students which is the total number of students with scores >70 / total students
passing_grade = 70
#First for reading
pass_reading = len(school_data_complete[school_data_complete['reading_score'] >= passing_grade])
pass_reading_per = pass_reading/number_of_students * 100
#Then for math
pass_math = len(school_data_complete[school_data_complete['math_score'] >= passing_grade])
pass_math_per = pass_math/number_of_students * 100
#Now it wants the average passing to be the average of the two averages
pass_overall_per = (pass_math_per + pass_reading_per)/2

In [7]:
#Next I am going to add all of these data points int a dictionary that will be turned into a dataframe
district_summary_dict = {"Total Schools":number_of_schools,"Total Students":number_of_students,"Total Budget":sum_budget,
                         "Average Math Score":average_math_score,"Average Reading Score":average_reading_score,
                         "% Passing Math":pass_math_per,"% Passing Reading":pass_reading_per,
                         "% Overall Passing Rate":average_overall_score}
district_summary_df = pd.DataFrame(district_summary_dict, index=[0])

In [8]:
#2 of the cells need to be cleaned up a little bit, the total students and total budget
#This can be achieved using the .map function
district_summary_df['Total Students'] = district_summary_df['Total Students'].map("{:,}".format)
district_summary_df['Total Budget'] = district_summary_df['Total Budget'].map("${:,.2f}".format)
district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,80.431606


In [9]:
#For the School summary section I can use the raw school data as a starting point, i just need to make the School name the index
school_summary_df = school_data_raw.copy().set_index("school_name")
school_summary_drop_df = school_summary_df.drop(columns=["School ID"])
school_summary_drop_df.columns = ['School type', 'Total Students', 'Total School Budget']

#Now that I have the base dateframe, i should make a new copy and then add the column for per student budget
school_summary_base = school_summary_drop_df.copy()
school_summary_base["Per Student Budget"] = school_summary_base["Total School Budget"]/school_summary_base["Total Students"]
school_summary_base.head()

,School type,Total Students,Total School Budget,Per Student Budget
school_name,,,,
Huang High School,District,2917,1910635,655.0
Figueroa High School,District,2949,1884411,639.0
Shelton High School,Charter,1761,1056600,600.0
Hernandez High School,District,4635,3022020,652.0
Griffin High School,Charter,1468,917500,625.0


In [10]:
#Now That I have a good starting base dataframe, I just need to make some series for the remaineder
#First to get the average math and reading scores grouped by school
math_group_df = school_data_complete.groupby('school_name')['math_score'].mean()
reading_group_df = school_data_complete.groupby('school_name')['reading_score'].mean()

In [11]:
#To find the percentage that are passing, I first need to get a dataframe each with a filter that only has scores above 70
math_filter_df = school_data_complete[school_data_complete['math_score'] >= 70]
reading_filter_df = school_data_complete[school_data_complete['reading_score'] >= 70]

#Then i can group those and get a count of students by school
math_pass_df = math_filter_df.groupby('school_name')['math_score'].count()
reading_pass_df = reading_filter_df.groupby('school_name')['reading_score'].count()

In [12]:
#Now I should be able to take new base dataframe, and create new columns and set them equal to these series
#First up are the average scores
school_summary_base['Average Math Score'] = math_group_df
school_summary_base['Average Reading Score'] = reading_group_df

#Then to calculate and clean up the passing percentage
school_summary_base['% Passing Math'] = (math_pass_df/school_summary_base['Total Students'])*100
school_summary_base['% Passing Reading'] = (reading_pass_df/school_summary_base['Total Students'])*100

#Finally the overall passing percentage is the the average of the pass per for math and reading
school_summary_base['% Overall Passing Rate'] = (school_summary_base['% Passing Math']+school_summary_base['% Passing Reading'])/2

#as a final cleaning step, both budget columns need to be converted to dollar signs
#before i do that i need to store the per student budget in a variable that i can use later
school_budget_per_student = school_summary_base[['Per Student Budget']]
school_summary_base['Total School Budget'] = school_summary_base['Total School Budget'].map("${:,.2f}".format)
school_summary_base['Per Student Budget'] = school_summary_base['Per Student Budget'].map("${:,.2f}".format)

In [14]:
#Now that all the information is collected up, its time to display the top 5 performing schools
school_sum_top_overall = school_summary_base.sort_values(by=["% Overall Passing Rate"], ascending=False)
school_sum_top_overall.head()

,School type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,95.203679


In [15]:
#Now that all the information is collected up, its time to display the top 5 performing schools
school_sum_bottom_overall = school_summary_base.sort_values(by=["% Overall Passing Rate"])
school_sum_bottom_overall.head()

,School type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


In [16]:
#For this next part we want to breaks the scores up not only by school but by grade
#for this it wants us to break it down by creating a series for each grade, group the schools, then combine them
#first to make the dataframes that are filtered by grade
ninth_grade = student_data_raw[student_data_raw['grade'] == "9th"]
tenth_grade = student_data_raw[student_data_raw['grade'] == "10th"]
elfth_grade = student_data_raw[student_data_raw['grade'] == "11th"]
zwolfth_grade = student_data_raw[student_data_raw['grade'] == "12th"]

#Then to take all those filtered df's and group then by school and take the average
ninth_grade_school = ninth_grade.groupby(ninth_grade['school_name']).mean()
tenth_grade_school = tenth_grade.groupby(tenth_grade['school_name']).mean()
elfth_grade_school = elfth_grade.groupby(elfth_grade['school_name']).mean()
zwolfth_grade_school = zwolfth_grade.groupby(zwolfth_grade['school_name']).mean()

#Whats left are filtered, group'd by dataframes for each grade. now I need to put all the correct data into a new df
math_grade_summary = pd.DataFrame({"9th":ninth_grade_school['math_score'],
                                   "10th":tenth_grade_school['math_score'],
                                   "11th":elfth_grade_school['math_score'],
                                   "12th":zwolfth_grade_school['math_score']})
math_grade_summary

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [17]:
#Then to do the same thing but for reading
reading_grade_summary = pd.DataFrame({"9th":ninth_grade_school['reading_score'],
                                      "10th":tenth_grade_school['reading_score'],
                                      "11th":elfth_grade_school['reading_score'],
                                      "12th":zwolfth_grade_school['reading_score']})
reading_grade_summary

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [18]:
#Next section wants us to add passing rate by various metrics such as school size, spending, and type
#First thing to do is have a new dataframe
base_binning_df = school_summary_base.copy()

In [19]:
#Now for a dataframe specific to the per student budget
holder_df = base_binning_df[['Average Math Score','Average Reading Score',
                             '% Passing Math','% Passing Reading','% Overall Passing Rate']]
frame = [school_budget_per_student,holder_df]
budget_binning_df = pd.concat(frame,axis=1)

#Next to set up the bins, I'm just going to use the recommended defaults
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

#then to execute the cut
budget_binning_df["Spending Range"] = pd.cut(budget_binning_df['Per Student Budget'], spending_bins,
#budget_binning_df["Spending Range"] = pd.cut(school_budget_per_student['Per Student Budget'], spending_bins,
                                             labels=group_names, include_lowest=True)

#Now that we have the column of bins, we can group by that column
spending_range_df = budget_binning_df.groupby(budget_binning_df['Spending Range']).mean()
spending_range_df = spending_range_df.drop("Per Student Budget",axis=1)
spending_range_df.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Range,,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


In [20]:
#Next we want to do this exact same thing but for school size
#First create the dataframe
size_binning_df = base_binning_df[['Total Students','Average Math Score','Average Reading Score',
                                     '% Passing Math','% Passing Reading','% Overall Passing Rate']]
#Next to set up the bins, I'm just going to use the recommended defaults
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
#then to execute the cut
size_binning_df["School Size"] = pd.cut(size_binning_df['Total Students'], size_bins,
                                             labels=group_names, include_lowest=True)

#Now that we have the column of bins, we can group by that column
size_range_df = size_binning_df.groupby(size_binning_df['School Size']).mean()
size_range_df = size_range_df.drop("Total Students",axis=1)
size_range_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


In [21]:
#One more time for the people in the... for school type. 
type_binning_df = base_binning_df[['School type','Average Math Score','Average Reading Score',
                                     '% Passing Math','% Passing Reading','% Overall Passing Rate']]
#For this one we can just go straight to groupby
type_range_df = type_binning_df.groupby(type_binning_df['School type']).mean()
type_range_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
